# Engine to generate samples for a basic blackjack game with a finite deck
## Resplit aces
## Double after splitting
## No surrender

In [ ]:
# Import numpy and pyplot, set random seed and figure size

import time, os, datetime, gc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 7]

## Pack Class

Define the pack class.  This contains the data on the shoe including total, running, and true counts

In [ ]:
# pack class; input: number of decks, penetration (how small the pack can get in terms of numbers of decks before shuffling)

class pack(object):
    
    def __init__(self, num):
        self.num = num

    
    # initiate the pack dictionary
    
    current_pack = {1:1}
    
    # fresh_pack dictionary for num decks
    
    def fresh_pack(self):
        return {
                2 : np.int64(4*(self.num)), 3 : np.int64(4*(self.num)), 4 : np.int64(4*self.num), 
                5 : np.int64(4*self.num), 6 : np.int64(4*self.num), 7 : np.int64(4*self.num), 
                8 : np.int64(4*self.num), 9 : np.int64(4*self.num), 10 : np.int64(16*self.num), 
                'A' : np.int64(4*self.num)
                }
    
    # shuffle to reset current_pack to fresh_pack
    

    
    # deal out card from cp (current pack)
    
    def deal(self, cp, card):
        cp[card] = np.int64(cp[card] - 1)
        return cp
        
    def card(self, cp):
        # get total number of cards in currentpack
        t = np.int64(sum(cp.values()))
 #       delta = {i: cp[i]/t for i in cp.keys()}
        # enforce proper chances based on distribution of current deck
        x = np.random.rand()
        if x < max(cp[2], 0)/ sum(cp.values()):
            return 2
        elif x < (max(cp[2],0) + max(cp[3],0))/ sum(cp.values()):
            return 3
        elif x < (max(cp[2],0) + max(cp[3],0) + max(cp[4],0))/ sum(cp.values()):
            return 4
        elif x < (max(cp[2],0) + max(cp[3],0) + max(cp[4],0) + max(cp[5],0))/ sum(cp.values()):
            return 5
        elif x < (max(cp[2],0) + max(cp[3],0) + max(cp[4],0) + max(cp[5],0) + max(cp[6],0))/ sum(cp.values()):
            return 6
        elif x < (max(cp[2],0) + max(cp[3],0) + max(cp[4],0) + max(cp[5],0) + max(cp[6],0) + max(cp[7],0))/ sum(cp.values()):
            return 7
        elif x < (max(cp[2],0) + max(cp[3],0) + max(cp[4],0) + max(cp[5],0) + max(cp[6],0) + max(cp[7],0) + max(cp[8],0))/ sum(cp.values()):
            return 8
        elif x < (max(cp[2],0) + max(cp[3],0) + max(cp[4],0) + max(cp[5],0) + max(cp[6],0) + max(cp[7],0) + max(cp[8],0) + max(cp[9],0))/ sum(cp.values()):
            return 9
        elif x < (max(cp[2],0) + max(cp[3],0) + max(cp[4],0) + max(cp[5],0) + max(cp[6],0) + max(cp[7],0) + max(cp[8],0) + max(cp[9],0) + max(cp[10],0))/ sum(cp.values()):
            return 10
        else:
            return 'A'
        
    # running count
        
    def rc(self, cp):
        # number of low cards dealt out
        num_lows = np.int64(self.num*20 - cp[2] - cp[3] - cp[4] - cp[5] - cp[6])
        # number of high cards dealt out
        num_highs = np.int64(self.num*20 - cp[10] - cp['A'])
        return np.int64(num_lows - num_highs)
    
    # true count (to nearest half-deck)
    
    def tc(self, cp):
        # get total number of cards in currentpack
        t = sum(cp.values())
        # number of decks remaining
        if t > 26:
            d = np.floor(2*t/52)/2
        else: d = 0.5
        if self.rc(cp) >= 0:
            return np.int64(np.floor(self.rc(cp)/d))
        else:
            return np.int64((-1)*np.floor(-self.rc(cp)/d))
        
  # get total number of cards in currentpack  

    def card_total(self, cp):
        return np.int64(sum(cp.values()))
        
        
        
    
    
        
    

Some useful functions for the engine: move Aces to end of hand, compute hand value

In [ ]:


# possible sums of a hand with aces

def psums(hand):
    aceless = [x for x in hand if x != 'A']
    return np.array([sum(aceless) + i*10 + hand.count('A') for i in range(hand.count('A') + 1)])


    

# gives value of hand
    
def hand_value(hand):
    if 'A' in hand:
        if min(psums(hand)) > 21:  # if you bust even with all aces = 1
            return np.min(psums(hand))
        else:
            return np.max(psums(hand)[psums(hand) <= 21])
    else:
            return sum(hand)  

## Naive Player Class

The next cell defines the naive player class.  This player only hits if there is no chance of bust.

In [ ]:
# naive player class. inputs: hand dealt, dealer top card, bet, spots, shoe.


class naive_player(object):
    
    def __init__(self, hands, top, bet, spots, shoe):
        self.hands = hands
        self.top = top
        self.bet = bet
        self.spots = spots
        self.shoe = shoe
        
    def Spots(self):
        return self.spots
    
    def betting(self):
        return self.bet
    
#     insurance = False

    def hit(self, i):
        new_card = self.shoe.card(self.shoe.current_pack)
        self.shoe.deal(self.shoe.current_pack, new_card)
        # print("Player hits, receives card ", new_card, '. Hand number', i+1,  self.hands[i], "becomes")
        self.hands[i] = self.hands[i] + [new_card]
        
    def stand(self):
        return [False]*len(self.hands)
        
    def result(self):
        return [[]]*len(self.hands)
    
    def is_soft(self,i):
        if 'A' in self.hands[i] and np.min(psums(self.hands[i])) <= 10:
            return True
        else: 
            return False
        
        
    def p_turn(self, D, result, j, stand, game, split_count, double_down):
        stand = self.stand()
        split_count = 0
        # print("Dealer's top card: ", self.top)
        if game.d_result == ['DBJ']:
            stand = [True]*len(self.hands)
            for i in range(len(self.hands)):
            # check for player BJs
                if hand_value(self.hands[i]) == 21 and len(self.hands[i]) == 2:
                    result[i] = ['PBJ']
            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
            # print('True count: ', self.shoe.tc(self.shoe.current_pack))    
            # print("Player's result(s):", result)
            return(result)
        else:
            for i in range(j,len(self.hands)):
                # check for BJs
                if hand_value(self.hands[i]) == 21 and len(self.hands[i]) == 2:
                    result[i] = ['PBJ']
                    stand[i] = True
                else:
                    self.p_strat(i, stand, result,D,split_count, game)
        
        
        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
        # print('True count: ', self.shoe.tc(self.shoe.current_pack))    
        # print("Player's result(s):", result)
        return(result)
    
    def p_strat(self, i, stand, result,D, split_count, game):
        while not stand[i]:
            if self.is_soft(i):
                # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                self.hit(i)
                # print(self.hands[i])
                # print("with value", hand_value(self.hands[i]))
            elif hand_value(self.hands[i]) <= 11:
                # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                self.hit(i)
                # print(self.hands[i])
                # print("with value", hand_value(self.hands[i]))
            else:
                # print("Player stands.")
                result[i] = ['stand']
                stand[i] = True
        


## Copycat Player Class

Next cell defines the copycat player class.  This player plays just like the dealer.

In [ ]:
# copycat player class. inputs: hand dealt, dealer top card. this is a player who plays just like the dealer

class copycat_player(object):
    
    def __init__(self, hands, top, bet, spots, shoe):
        self.hands = hands
        self.top = top
        self.bet = bet
        self.spots = spots
        self.shoe = shoe
        
    def Spots(self):
        return self.spots
    
    def betting(self):
        return self.bet
    
#     insurance = False

    def hit(self, i):
        new_card = self.shoe.card(self.shoe.current_pack)
        self.shoe.deal(self.shoe.current_pack, new_card)
        # print("Player hits, receives card ", new_card, '. Hand number', i+1,  self.hands[i], "becomes")
        self.hands[i] = self.hands[i] + [new_card]
        
    def stand(self):
        return [False]*len(self.hands)
        
    def result(self):
        return [[]]*len(self.hands)
        
        
    def p_turn(self, D, result, j, stand, game, split_count, double_down):
        stand = self.stand()
        split_count = 0
        # print("Dealer's top card: ", self.top)
        if game.d_result == ['DBJ']:
            stand = [True]*len(self.hands)
            for i in range(len(self.hands)):
            # check for player BJs
                if hand_value(self.hands[i]) == 21 and len(self.hands[i]) == 2:
                    result[i] = ['PBJ']
            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
            # print('True count: ', self.shoe.tc(self.shoe.current_pack))    
            # print("Player's result(s):", result)
            return(result)
        else:
            for i in range(j,len(self.hands)):
                # check for BJs
                if hand_value(self.hands[i]) == 21 and len(self.hands[i]) == 2:
                    result[i] = ['PBJ']
                    stand[i] = True
                else:
                    self.p_strat(i, stand, result,D,split_count, game)
        
        
        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
        # print('True count: ', self.shoe.tc(self.shoe.current_pack))    
        # print("Player's result(s):", result)
        return(result)
    
    def p_strat(self, i, stand, result,D, split_count, game):
        while hand_value(self.hands[i]) < 17:
            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
            self.hit(i)
            # print(self.hands[i])
            # print("with value", hand_value(self.hands[i]))
        if hand_value(self.hands[i]) <= 21:
            result[i] = ['stand']
            stand[i] = True
        else:
            # print("Player busts")
            result[i] = ['bust']
            stand[i] = True
        

## Basic Player Class

The next cell contains the basic_player class.  Ths player bets flat and plays according to basic strategy

In [ ]:
# basic strategy player class. inputs: hand dealt (2D list), dealer top card (int), bet (list), number of spots (int)
#this is a player who bets flat and plays according to basic strategy

class basic_player(object):
    
    def __init__(self, hands, top, bet, spots, shoe):
        self.hands = hands
        self.top = top
        self.bet = bet
        self.spots = spots
        self.shoe = shoe
        
#     def wager(self):
#         return(1)
    
    
    
    def Spots(self):
        return self.spots
    
    def betting(self):
        return self.bet
    
#     insurance = False
    
    #take a hit

    def hit(self, i):
        new_card = self.shoe.card(self.shoe.current_pack)
        self.shoe.deal(self.shoe.current_pack, new_card)
        # print("Player hits, receives card ", new_card, '. Hand number', i+1,  self.hands[i], "becomes")
        self.hands[i] = self.hands[i] + [new_card]
    
    # double down
    
    def double(self,i, stand, double_down):
        new_card = self.shoe.card(self.shoe.current_pack)
        self.shoe.deal(self.shoe.current_pack, new_card)
        # print("Player doubles, receives card ", new_card, '. Hand number', i+1, self.hands[i], "becomes")
        self.hands[i] = self.hands[i] + [new_card]
        self.bet[i] = 2*self.bet[i]
        stand[i] = True
        double_down[i] = True
        
    # split a pair
    
    def split(self, i, stand, D, result, game, split_count, double_down):
        # print("Player splits hand number ", i+1)
        if self.hands[i][0] != 'A':
            self.hands[i][1] = self.shoe.card(self.shoe.current_pack)
            self.shoe.deal(self.shoe.current_pack, self.hands[i][1])
            self.hands.insert(i+1, [self.hands[i][0], self.shoe.card(self.shoe.current_pack)])
            self.shoe.deal(self.shoe.current_pack, self.hands[i+1][1])
            result.insert(i+1, [])
            double_down.insert(i+1, False)
            self.bet.insert(i+1, self.bet[0])
            stand.insert(i+1, False)
            # print("Player's hands are now ", self.hands)
            self.p_turn(D, result,i, stand, game, split_count, double_down)
        else:
            self.hands[i][0] = self.shoe.card(self.shoe.current_pack)
            self.shoe.deal(self.shoe.current_pack, self.hands[i][0])
            self.hands.insert(i+1, [self.shoe.card(self.shoe.current_pack), self.hands[i][1]])
            self.shoe.deal(self.shoe.current_pack, self.hands[i+1][0])
            stand[i] = True
            stand.insert(i+1, True)
            result.insert(i+1, [])
            double_down.insert(i+1, False)
            self.bet.insert(i+1, self.bet[0])
            # print("Player's hands are now ", self.hands)
            self.p_turn(D, result, i,stand, game, split_count, double_down)
        
    def split_test(self, i, split_count):
        if i <= split_count[0]:
            if split_count[0] <= 2:
                return True
            else:
                return False
        else:
            if split_count[1] <=2:
                return True
            else:
                return False
    
    def is_pair(self,i):
        if len(self.hands[i]) == 2 and self.hands[i][0] == self.hands[i][1]:
            return True
        else:
            return False
        
    def is_soft(self,i):
        if 'A' in self.hands[i] and np.min(psums(self.hands[i])) <= 10:
            return True
        else: 
            return False
        
        
    def stand(self):
        return [False]*len(self.hands)
        
    def result(self):
        return [[]]*len(self.hands)
       

    def p_turn(self, D, result, j, stand, game, split_count, double_down):
        stand = self.stand()
        # print("Dealer's top card: ", self.top)
        if game.d_result == ['DBJ']:
            stand = [True]*len(self.hands)
            for i in range(len(self.hands)):
            # check for player BJs
                if hand_value(self.hands[i]) == 21 and len(self.hands[i]) == 2:
                    result[i] = ['PBJ']
            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
            # print('True count: ', self.shoe.tc(self.shoe.current_pack))    
            # print("Player's result(s):", result)
            return(result)
        else:
            for i in range(j,len(self.hands)):
                # check for BJs
                if hand_value(self.hands[i]) == 21 and len(self.hands[i]) == 2:
                    result[i] = ['PBJ']
                    stand[i] = True
                else:
                    self.p_strat(i, stand, result,D,split_count, game, double_down)
        
        
        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
        # print('True count: ', self.shoe.tc(self.shoe.current_pack))    
        # print("Player's result(s):", result)
        return(result)
    
    def p_strat(self, i, stand, result,D, split_count, game, double_down):
        
        
        # print("Player's hand number ", i+1, " is ", self.hands[i])
        while not stand[i]:
# soft totals
            if self.is_soft(i) and self.hands[i] != ['A', 'A']:
                if hand_value(self.hands[i]) == 20:
                    result[i] = ['stand']
                    stand[i] = True
                if hand_value(self.hands[i]) == 19:
                    if self.top == 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            result[i] = ['stand']
                            stand[i] = True
                    else:
                        result[i] = ['stand']
                        stand[i] = True
                if hand_value(self.hands[i]) == 18:
                    if self.top != 'A' and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i, stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            result[i] = ['stand']
                            stand[i] = True
                    elif self.top != 'A' and self.top <= 8:
                        result[i] = ['stand']
                        stand[i] = True
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) == 17:
                    if self.top != 'A' and self.top >= 3 and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) in [15, 16]:
                    if self.top != 'A' and self.top >= 4 and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) in [13, 14]:
                    if self.top != 'A' and self.top >= 5 and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    
                    
                            
                        
                        
                        
# pairs                            
                    
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 'A':
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
            
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 9 and self.top in [2,3,4,5,6,8,9]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
            
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 8:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 7 and self.top in [2,3,4,5,6,7]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 6 and self.top in [2,3,4,5,6]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 4 and self.top in [5,6]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 3 and self.top in [2,3,4,5,6,7]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 2 and self.top in [2,3,4,5,6,7]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
                
                
                
 
    
# hard totals

            else:
                if hand_value(self.hands[i]) >= 17:
                    if hand_value(self.hands[i]) <= 21:
                        # print("Player stands.")
                        result[i] = ['stand']
                        stand[i] = True
                    else:
                        # print("Player busts")
                        result[i] = ['bust']
                        stand[i] = True
                if hand_value(self.hands[i]) <= 8:
                    # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                    # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                    self.hit(i)
                    # print(self.hands[i])
                    # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) >= 13 and hand_value(self.hands[i]) <= 16:
                    if self.top != 'A' and self.top <= 6:
                        # print("Player stands.")
                        result[i] = ['stand']
                        stand[i] = True
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) == 12:
                    if self.top != 'A' and self.top >= 4 and self.top<= 6:
                        # print("Player stands.")
                        result[i] = ['stand']
                        stand[i] = True
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) == 11:
                    if len(self.hands[i]) == 2:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.double(i,stand, double_down)
                        result[i] = ['stand']
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) == 10:
                    if self.top != 'A' and self.top <= 9:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) == 9:
                    if self.top != 'A' and self.top >=3 and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else: 
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    
        

In [ ]:
# function to cut off large values at 6

def max6(count):
    if count < 6:
        return count
    else: 
        return 6

## Card Counter Class

The next cell has the counter class.

In [ ]:
# card counter strategy player class. inputs: hands dealt (2D list), dealer top card (int), spread (dict), count (int)
# this player plays the way a card counter would based on a specific bet spread

class card_counter(object):
    
    def __init__(self, hands, top, shoe, bet, spread, opening_count):
        self.hands = hands
        self.top = top
        self.shoe = shoe
        self.bet = bet
        self.spread = spread
        self.opening_count = opening_count
        
    
    def Spots(self):
        return self.spread[max6(self.opening_count)][1]
   
    
    def betting(self):
        self.bet = [self.spread[max6(self.opening_count)][0]]*self.Spots()
    
#     insurance = False
    
    
    
    #take a hit

    def hit(self, i):
        new_card = self.shoe.card(self.shoe.current_pack)
        self.shoe.deal(self.shoe.current_pack, new_card)
        # print("Player hits, receives card ", new_card, '. Hand number', i+1,  self.hands[i], "becomes")
        self.hands[i] = self.hands[i] + [new_card]
    
    # double down
    
    def double(self,i, stand, double_down):
        new_card = self.shoe.card(self.shoe.current_pack)
        self.shoe.deal(self.shoe.current_pack, new_card)
        # print("Player doubles, receives card ", new_card, '. Hand number', i+1, self.hands[i], "becomes")
        self.hands[i] = self.hands[i] + [new_card]
        self.bet[i] = 2*self.bet[i]
        # print('Bets are now ', self.bet)
        double_down[i] = True
        stand[i] = True
        
        
    # split a pair
    
    def split(self, i, stand, D, result, game, split_count, double_down):
        # print("Player splits hand number ", i+1)
        if self.hands[i][0] != 'A':
            self.hands[i][1] = self.shoe.card(self.shoe.current_pack)
            self.shoe.deal(self.shoe.current_pack, self.hands[i][1])
            self.hands.insert(i+1, [self.hands[i][0], self.shoe.card(self.shoe.current_pack)])
            self.shoe.deal(self.shoe.current_pack, self.hands[i+1][1])
            result.insert(i+1, [])
            self.bet.insert(i+1, self.bet[0])
            stand.insert(i+1, False)
            double_down.insert(i+1, False)
            # print("Player's hands are now ", self.hands)
            # print('Bets are now ', self.bet)
            self.p_turn(D, result,i, stand, game, split_count, double_down)
        else:
            self.hands[i][0] = self.shoe.card(self.shoe.current_pack)
            self.shoe.deal(self.shoe.current_pack, self.hands[i][0])
            self.hands.insert(i+1, [self.shoe.card(self.shoe.current_pack), self.hands[i][1]])
            self.shoe.deal(self.shoe.current_pack, self.hands[i+1][0])
            stand[i] = True
            stand.insert(i+1, True)
            double_down.insert(i+1, False)
            result.insert(i+1, [])
            self.bet.insert(i+1, self.bet[0])
            # print("Player's hands are now ", self.hands)
            # print('Bets are now ', self.bet)
            self.p_turn(D, result, i, stand, game, split_count, double_down)
    
    def split_test(self, i, split_count):
        if i <= split_count[0]:
            if split_count[0] <= 2:
                return True
            else:
                return False
        else:
            if split_count[1] <=2:
                return True
            else:
                return False
        
        
    def is_pair(self,i):
        if len(self.hands[i]) == 2 and self.hands[i][0] == self.hands[i][1]:
            return True
        else:
            return False
        
    def is_soft(self,i):
        if 'A' in self.hands[i] and np.min(psums(self.hands[i])) <= 10:
            return True
        else: 
            return False
        
        
    def stand(self):
        return [False]*len(self.hands)
        
    def result(self):
        return [[]]*len(self.hands)
       

    def p_turn(self, D, result, j, stand, game, split_count, double_down):
        # print("Dealer's top card: ", self.top)
#         if self.top == 'A' and self.shoe.tc(self.shoe.current_pack) >= 3:
#             # print('Player takes insurance.')
#             insurance = True
        if game.d_result == ['DBJ']:
            stand = [True]*len(self.hands)
            for i in range(len(self.hands)):
            # check for player BJs
                if hand_value(self.hands[i]) == 21 and len(self.hands[i]) == 2:
                    result[i] = ['PBJ']
            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
            # print('True count: ', self.shoe.tc(self.shoe.current_pack))    
            # print("Player's result(s):", result)
            return(result)
        else:
            for i in range(j,len(self.hands)):
                # check for BJs
                if hand_value(self.hands[i]) == 21 and len(self.hands[i]) == 2:
                    result[i] = ['PBJ']
                    stand[i] = True
                else:
                    self.p_strat(i, stand, result,D,split_count, game, double_down)
        
        
        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
        # print('True count: ', self.shoe.tc(self.shoe.current_pack))    
        # print("Player's result(s):", result)
        return result
    
    def p_strat(self, i, stand, result,D, split_count, game, double_down):
        
        
        # print("Player's hand number ", i+1, " is ", self.hands[i])
        while not stand[i]:
            # update current count
            current_count = self.shoe.tc(self.shoe.current_pack)
            
# soft totals
            if self.is_soft(i) and self.hands[i] != ['A', 'A']:
                if hand_value(self.hands[i]) == 20:
                    result[i] = ['stand']
                    stand[i] = True
                if hand_value(self.hands[i]) == 19:
                    if self.top == 4 and current_count >= 3:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            result[i] = ['stand']
                            stand[i] = True
                    elif self.top == 5 and current_count >= 1:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            result[i] = ['stand']
                            stand[i] = True
                    elif self.top == 6 and current_count < 0:
                        result[i] = ['stand']
                        stand[i] = True
                    elif self.top == 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            result[i] = ['stand']
                            stand[i] = True
                    else:
                        result[i] = ['stand']
                        stand[i] = True
                if hand_value(self.hands[i]) == 18:
                    if self.top != 'A' and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i, stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            result[i] = ['stand']
                            stand[i] = True
                    elif self.top != 'A' and self.top <= 8:
                        result[i] = ['stand']
                        stand[i] = True
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) == 17:
                    if self.top == 2 and current_count >= 1:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    elif self.top != 'A' and self.top >= 3 and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) in [15, 16]:
                    if self.top != 'A' and self.top >= 4 and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                if hand_value(self.hands[i]) in [13, 14]:
                    if self.top != 'A' and self.top >= 5 and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    
                    
                            
                        
                        
                        
# pairs                            
                    
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 'A':
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 10 and self.top == 4 and current_count >= 6:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 10 and self.top == 5 and current_count >= 5:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break 
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 10 and self.top == 6 and current_count >= 4:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break  
            
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 9 and self.top in [2,3,4,5,6,8,9]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
            
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 8:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 7 and self.top in [2,3,4,5,6,7]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 6 and self.top in [2,3,4,5,6]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 4 and self.top in [5,6]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 3 and self.top in [2,3,4,5,6,7]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
            elif self.is_pair(i) and self.split_test(i, split_count) and self.hands[i][0] == 2 and self.top in [2,3,4,5,6,7]:
                if i <= split_count[0]:
                    split_count[0] += 1
                else:
                    split_count[-1] += 1
                self.split(i,stand,D, result, game, split_count, double_down)
                break
                
                
                
                
 
    
# hard totals

            else:
                if hand_value(self.hands[i]) >= 17:
                    if hand_value(self.hands[i]) <= 21:
                        # print("Player stands.")
                        result[i] = ['stand']
                        stand[i] = True
                    else:
                        # print("Player busts")
                        result[i] = ['bust']
                        stand[i] = True
                elif hand_value(self.hands[i]) == 16 and self.top == 9 and current_count >= 4:
                    # print("Player stands.")
                    result[i] = ['stand']
                    stand[i] = True
                elif hand_value(self.hands[i]) == 16 and self.top == 10 and current_count >= 0:
                    # print("Player stands.")
                    result[i] = ['stand']
                    stand[i] = True
                elif hand_value(self.hands[i]) == 16 and self.top == 'A' and current_count >= 3:
                    # print("Player stands.")
                    result[i] = ['stand']
                    stand[i] = True
                elif hand_value(self.hands[i]) == 15 and self.top == 10 and current_count >= 4:
                    # print("Player stands.")
                    result[i] = ['stand']
                    stand[i] = True
                elif hand_value(self.hands[i]) == 15 and self.top == 'A' and current_count >= 5:
                    # print("Player stands.")
                    result[i] = ['stand']
                    stand[i] = True
                elif hand_value(self.hands[i]) >= 13 and hand_value(self.hands[i]) <= 16:
                    if self.top != 'A' and self.top <= 6:
                        # print("Player stands.")
                        result[i] = ['stand']
                        stand[i] = True
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) == 13 and self.top == 2 and current_count <= -1:
                    # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                    # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                    self.hit(i)
                    # print(self.hands[i])
                elif hand_value(self.hands[i]) == 12 and self.top == 2 and current_count >= 3:
                    # print("Player stands.")
                    result[i] = ['stand']
                    stand[i] = True
                elif hand_value(self.hands[i]) == 12 and self.top == 3 and current_count >= 2:
                    # print("Player stands.")
                    result[i] = ['stand']
                    stand[i] = True
                elif hand_value(self.hands[i]) == 12 and self.top == 4 and current_count <= 0:
                    # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                    # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                    self.hit(i)
                    # print(self.hands[i])
                elif hand_value(self.hands[i]) == 12:
                    if self.top != 'A' and self.top >= 4 and self.top<= 6:
                        # print("Player stands.")
                        result[i] = ['stand']
                        stand[i] = True
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) == 11:
                    if len(self.hands[i]) == 2:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.double(i,stand, double_down)
                        result[i] = ['stand']
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) == 10 and self.top == 10 and current_count >= 4:
                    if len(self.hands[i]) == 2:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.double(i,stand, double_down)
                        result[i] = ['stand']
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) == 10 and self.top == 'A' and current_count >= 3:
                    if len(self.hands[i]) == 2:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.double(i,stand, double_down)
                        result[i] = ['stand']
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) == 10:
                    if self.top != 'A' and self.top <= 9:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) == 9 and self.top == 2 and current_count >= 1:
                    if len(self.hands[i]) == 2:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.double(i,stand, double_down)
                        result[i] = ['stand']
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) == 9 and self.top == 7 and current_count >= 3:
                    if len(self.hands[i]) == 2:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.double(i,stand, double_down)
                        result[i] = ['stand']
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) == 9:
                    if self.top != 'A' and self.top >=3 and self.top <= 6:
                        if len(self.hands[i]) == 2:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.double(i,stand, double_down)
                            result[i] = ['stand']
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                        else:
                            # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                            # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                            self.hit(i)
                            # print(self.hands[i])
                            # print("with value", hand_value(self.hands[i]))
                    else: 
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) == 8 and self.top == 6 and current_count >= 2:
                    if len(self.hands[i]) == 2:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.double(i,stand, double_down)
                        result[i] = ['stand']
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                    else:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit(i)
                        # print(self.hands[i])
                        # print("with value", hand_value(self.hands[i]))
                elif hand_value(self.hands[i]) <= 8:
                    # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                    # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                    self.hit(i)
                    # print(self.hands[i])
                    # print("with value", hand_value(self.hands[i]))        
                
                    
        

## Dealer Class

The next cell contains the dealer class.  The dealer can play S17 or H17 based on the `seventeen` parameter.

In [ ]:
# dealer class; input: hand
class dealer(object):
    def __init__(self, hand, shoe, seventeen):
        self.hand = hand
        self.shoe = shoe
        self.seventeen = seventeen
        
# If seventeen = 'S17', dealer stands on soft seventeen
# If seventeen = 'H17', dealer hits on soft seventeen

    def is_soft(self):
        if 'A' in self.hand and np.min(psums(self.hand)) <= 10:
            return True
        else: 
            return False
    
    def hit(self):
        new_card = self.shoe.card(self.shoe.current_pack)
        self.shoe.deal(self.shoe.current_pack, new_card)
        # print("Dealer hits, receives card ", new_card, '. ', self.hand, "becomes")
        self.hand = self.hand + [new_card]
        
    def result(self):
        return []
    
        
    def d_turn(self,P,p_result):
        # print("Dealer's turn:")
        result = self.result()
        if hand_value(self.hand) == 21 and len(self.hand) == 2:
            result = game.d_result
        elif p_result == [['PBJ']]*len(P.hands):
            result = ['stand']
        else:
            if self.seventeen == 'S17':
                while hand_value(self.hand) < 17:
                    # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                    # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                    self.hit()
                    # print(self.hand) 
                    # print(hand_value(self.hand))
                if hand_value(self.hand) <= 21:
                    result = ['stand']
                else:
                    result = ['bust']
            if self.seventeen == 'H17':
                while result == []:
                    if hand_value(self.hand) < 17:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit()
                        # print(self.hand) 
                        # print(hand_value(self.hand)) 
                    elif self.is_soft() and hand_value(self.hand) == 17:
                        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
                        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
                        self.hit()
                        # print(self.hand) 
                        # print(hand_value(self.hand))
                    elif hand_value(self.hand) <= 21:
                        result = ['stand']
                    else:
                        result = ['bust']
                        
                

        # print('Running count: ', self.shoe.rc(self.shoe.current_pack))            
        # print('True count: ', self.shoe.tc(self.shoe.current_pack))
        # print("Dealer's result: ", result)
        return result

## Game-play class

The next cell contains the class for game play

In [ ]:
class play(object):
    def __init__(self, shoe, pen, strategy, seventeen, BJpays, spread):
        self.shoe = shoe
        self.pen = pen
        self.strategy = strategy
        self.seventeen = seventeen
        self.BJpays = BJpays
        self.spread = spread
        
    def fill_shoe(self):
        self.shoe.current_pack = self.shoe.fresh_pack()
        
    def BJratio(self):
        return self.BJpays[0]/self.BJpays[1]

# initiate player and dealer

    def PD(self):
        
        D = dealer([],self.shoe)
        
        if self.strategy == 'cc':
        
            P = copycat_player([],0,[1],1, self.shoe)
            
        if self.strategy == 'basic':
        
            P = basic_player([],0,[1],1, self.shoe)
            
        if self.strategy == 'counter':
            pass

# deal the cards            
            
    def start_game(self, P, D):
        
        # deal out first card to player in all spots
        for i in range(P.spots):
            P.hands = P.hands + [[self.shoe.card(self.shoe.current_pack)]]
            self.shoe.deal(self.shoe.current_pack, P.hands[i][0])
        # deal first card to dealer
        D.hand = D.hand + [self.shoe.card(self.shoe.current_pack)]
        self.shoe.deal(self.shoe.current_pack, D.hand[0])
        # deal out second card to player in all spots
        for i in range(P.spots):
            P.hands[i] = P.hands[i] + [self.shoe.card(self.shoe.current_pack)]
            self.shoe.deal(self.shoe.current_pack, P.hands[i][1])
        # deal second card to dealer
        D.hand = D.hand + [self.shoe.card(self.shoe.current_pack)]
        self.shoe.deal(self.shoe.current_pack, D.hand[1])
        
        
# check if the dealer has BJ
    d_result = []

    def BJ_check(self, D):
        if hand_value(D.hand) == 21 and len(D.hand) == 2:
            # print("Dealer has Blackjack")
            d_result = ['DBJ']
        else:
            d_result =[]
        # print(d_result)
            
    
        

    def round(self):
       
        # check if a shuffle is needed
        if self.shoe.card_total(self.shoe.current_pack) < self.pen*52:
            # print('**SHUFFLING**')
            self.shoe.current_pack = self.shoe.fresh_pack()
            
        # record opening count for counter's betting
        
        
           
            
        
        
        # initiate players
        D = dealer([], self.shoe, self.seventeen)
        
        if self.strategy == 'naive':
            
            P = naive_player([],0,[1],1, self.shoe)
        
        if self.strategy == 'cc':
        
            P = copycat_player([],0,[1],1, self.shoe)
            
        if self.strategy == 'basic':
        
            P = basic_player([],0,[1],1, self.shoe)
            
        if self.strategy == 'counter':
            opening_count = self.shoe.tc(self.shoe.current_pack)
            P = card_counter([],0,self.shoe,[],self.spread, opening_count)
            
            # card counter doesn't play if the count is too low
        
            if self.shoe.tc(self.shoe.current_pack) <= -2:
                # print('Player requests a shuffle/changes tables \n because the opening count is too low.')
                # print('**SHUFFLING**')
                self.shoe.current_pack = self.shoe.fresh_pack() 
                P.opening_count = self.shoe.tc(self.shoe.current_pack)
                opening_count = self.shoe.tc(self.shoe.current_pack)
            
            
            opening_count = self.shoe.tc(self.shoe.current_pack)
            
            
            # print('Opening count is ', opening_count)
            
            
            # print('Player bets ', self.spread[max6(opening_count)][0], ' on ', self.spread[max6(opening_count)][1], ' spots.')
            
            
            P.bet = [self.spread[max6(opening_count)][0]]*self.spread[max6(opening_count)][1]
        
        opening_count = self.shoe.tc(self.shoe.current_pack)
        # deal out cards
        # deal out first card to player in all spots
        for i in range(P.Spots()):
            P.hands = P.hands + [[self.shoe.card(self.shoe.current_pack)]]
            self.shoe.deal(self.shoe.current_pack, P.hands[i][0])
        # deal first card to dealer
        D.hand = D.hand + [self.shoe.card(self.shoe.current_pack)]
        self.shoe.deal(self.shoe.current_pack, D.hand[0])
        # deal out second card to player in all spots
        for i in range(P.Spots()):
            P.hands[i] = P.hands[i] + [self.shoe.card(self.shoe.current_pack)]
            self.shoe.deal(self.shoe.current_pack, P.hands[i][1])
        # deal second card to dealer
        D.hand = D.hand + [self.shoe.card(self.shoe.current_pack)]
        self.shoe.deal(self.shoe.current_pack, D.hand[1])
        
        # set the top card
        
        P.top = D.hand[1]
        
        # # print the hands
        
        # print("Player's hand: " + str(P.hands))
        # print("Dealer's hand: " + str(D.hand))
        
        # check shoe
        
        # print("Shoe status: ", self.shoe.current_pack)
        
        # initialize player's bets
        
        P.betting()
        
        # initialize p_result, split_count, and double_down lists

        p_result = [[]]*len(P.hands)
        split_count = [0]*len(P.hands)
        double_down = [False]*len(P.hands)
        
        # if dealer's top card is  an ace, offer insurance; first initialize insurance as False
        
        insurance = False
        
        if P.top == 'A':
            if self.strategy == 'counter' and opening_count >=3:
                insurance = True
        
        # check if the dealer has BJ
        
        if len(D.hand) == 2 and hand_value(D.hand) == 21:
            d_result = ['DBJ']
            for i in range(len(P.hands)):
                # check for player BJs
                if hand_value(P.hands[i]) == 21 and len(P.hands[i]) == 2:
                    p_result[i] = ['PBJ']
        
        
        else: # If dealer doesn't have BJ player takes turn
            
            
            stand = P.stand()
            
            p_result = P.p_turn(D,p_result,0, stand, self, split_count, double_down)
        
            # check if player busted all hands
        
            if p_result == [['bust']]*len(P.hands):
                # print("Player busted. Dealer stands.")
                d_result = ['stand']
            else:  # if player didn't bust all hands, dealer takes turn
                d_result = D.d_turn(P,p_result)
        
        

        
        # # print results
        
        # print("Player's result(s): ", p_result)
        # print("Dealer's result: ", d_result)
        # print("Player's bet(s): ", P.bet)
        p_winnings_by_hand = self.outcome(p_result, d_result, P,D, double_down, insurance)
        # print("Player's winnings by hand: ", p_winnings_by_hand)
        # print("Player's winnings this round: ", sum(p_winnings_by_hand))
        # print("Shoe status: ", self.shoe.current_pack)
        # print("Number of cards in shoe: ", sum(self.shoe.current_pack.values()))
        winnings = sum(p_winnings_by_hand)
        return winnings, {'opening_count': opening_count,\
                          'cards_left': self.shoe.card_total(self.shoe.current_pack),\
                          'spots': P.Spots(), \
                          'player_bets': P.bet, 'double_down': double_down, \
                          'player_hands': P.hands,'dealer_hand': D.hand, \
                          'player_results': p_result, 'dealer_result': d_result, \
                          'insurance': insurance, \
                          'player_winnings_by_hand': p_winnings_by_hand, \
                          'total_round_winnings': sum(p_winnings_by_hand)}
        
        
        

        
        
        

        

        
        

    def outcome(self,p_result,d_result,P, D, double_down, insurance):
        #initiate winnings_by_hand list
        winnings_by_hand = [0]*len(P.hands)
        
        # if Dealer has BJ:
        if d_result == ['DBJ']:
            for i in range(len(P.hands)):
                if p_result[i] != ['PBJ']:
                    winnings_by_hand[i] = -1*P.bet[i] + insurance * (1 - 0.5 * double_down[i]) * P.bet[i]
        elif p_result == [['PBJ']] * len(P.hands):
            winnings_by_hand = [self.BJratio()*P.bet[0]]*len(P.hands)
        elif d_result == ['bust']:
            for i in range(len(P.hands)):
                if p_result[i] == ['bust']:
                    winnings_by_hand[i] = -1*P.bet[i] - insurance * 0.5 * (1 - 0.5 * double_down[i]) * P.bet[i]
                elif p_result[i] == ['PBJ']:
                    winnings_by_hand[i] = self.BJratio()*P.bet[i] - insurance * 0.5 * (1 - 0.5 * double_down[i]) * P.bet[i]
                else:
                    winnings_by_hand[i] = 1*P.bet[i] - insurance * 0.5 * (1 - 0.5 * double_down[i]) * P.bet[i]
        else:
            for i in range(len(P.hands)):
                if p_result[i] == ['bust']:
                    winnings_by_hand[i] = -1*P.bet[i] - insurance * 0.5 * (1 - 0.5 * double_down[i]) * P.bet[i]
                elif p_result[i] == ['PBJ']:
                    winnings_by_hand[i] = self.BJratio()*P.bet[i] - insurance * 0.5 * (1 - 0.5 * double_down[i]) * P.bet[i]
                else:
                    if hand_value(D.hand) == hand_value(P.hands[i]):
                        winnings_by_hand[i] = 0 - insurance * 0.5 * (1 - 0.5 * double_down[i]) * P.bet[i]
                    elif hand_value(D.hand) < hand_value(P.hands[i]):
                        winnings_by_hand[i] = 1*P.bet[i] - insurance * 0.5 * (1 - 0.5 * double_down[i]) * P.bet[i]
                    else:
                        winnings_by_hand[i] = -1*P.bet[i] - insurance * 0.5 * (1 - 0.5 * double_down[i]) * P.bet[i]
        return winnings_by_hand

## Samples class

The next cell contains the class for running samples.  It is customizable in that you can choose the number of decks, the penetration, the strategy for the players.  More options to come like the dealer's rules.  Also right now it only generates a list of total winnings, but later it will generate a dataframe with more data including each hand, counts, etc.

In [ ]:
# class for running samples

class sample(object):
    
    def __init__(self, shoe, pen, strategies, seventeen, num_players, num_rounds, BJpays, spread):
        self.shoe = shoe
        self.pen = pen
        self.strategies = strategies
        self.seventeen = seventeen
        self.num_players = num_players
        self.num_rounds = num_rounds
        self.BJpays = BJpays
        self.spread = spread
    
    # method for sampling N rounds
    
    def sample_rounds(self, strategy):
        
        game = play(self.shoe, self.pen, strategy, self.seventeen, self.BJpays)
        game.fill_shoe()
        
        winnings_list = []
        count_list = []
        
        for i in range(self.num_rounds):
            # print('**ROUND** ', i+1)
            count_list = count_list + [self.shoe.tc(self.shoe.current_pack)]
            winnings_list = winnings_list + [game.round()[0]]
            
        total_winnings_list = [sum(winnings_list[:i]) for i in range(len(winnings_list))]
        
        return np.array(total_winnings_list), np.array(winnings_list), np.array([count_list])
    
    # method for sampling M players each playing N rounds (heads up)
    
    def sample_players(self, strategy):
        
        return np.array([self.sample_rounds(strategy)[0] for i in range(self.num_players)]).transpose()
    
    # retrieve samples and statistics
    
    def retrieve(self):
        
        winnings_dict = {}
        counts_dict = {}
        #counts_delta_dict = {}
        
        for index, strategy in enumerate(self.strategies):
            
            
            np.random.seed(8)
            # print("Sampling ", strategy)
            samples = self.sample_rounds(strategy)
            total_winnings = samples[0]
            winnings_dict[strategy] = samples[1]
            counts_dict[strategy] = samples[2]
            #counts_delta_dict[strategy] = np.array([sum(counts_dict[strategy][:i]) for i in range(len(counts_dict[strategy]))])
#             final_winnings = np.array(samples[-1])
#             # print("Final winnings by player for strategy ", strategy, ": ", final_winnings)
#             # print("Average final winnings for strategy ", strategy, ": ", np.mean(final_winnings))
#             # print("Average winnings per round for strategy ", strategy, ": ", np.mean(final_winnings)/self.num_rounds)
            plt.plot(total_winnings, label=strategy)
        
            
        
        plt.title('Winnings for copycat, basic, and counter strategies with identical decks')
        plt.xlabel('Rounds')
        plt.ylabel('Total winnings')
        plt.legend(loc='lower left')
        plt.savefig('%s_%s_by_%s_winnings.pdf' % (str(self.strategies), self.num_players, self.num_rounds))
        plt.show()
        
        for strategy in self.strategies:
            plt.scatter(x=winnings_dict[strategy], y=counts_dict[strategy])
            plt.title(strategy)
            plt.xlabel('round winnings')
            plt.ylabel('opening count')
            plt.show()
            
#             plt.scatter(x=winnings_dict[strategy], y=counts_delta_dict[strategy])
#             plt.title(strategy)
#             plt.xlabel('round winnings')
#             plt.ylabel('change in count')
#             plt.show()
    def data_table(self):
        
        
        
        strategy_list = []
        decks_list = []
        pen_list = []
        BJpays_list = []
        seventeen_list = []
        opening_count_list = []
        cards_left_list =[]
        spots_list = []
        bets_list = []
        double_list = []
        p_hands_list = []
        d_hands_list = []
        p_results_list = []
        d_result_list = []
        insurance_list = []
        wbh_list = []
        trw_list = []
        
        
        
        
        for strategy in self.strategies:
            np.random.seed(80)
            game = play(self.shoe, self.pen, strategy, self.seventeen, self.BJpays, self.spread)
            game.fill_shoe()
            for i in range(self.num_rounds):
                outcome = game.round()[1]
                strategy_list.append(strategy)
                decks_list.append(self.shoe.num)
                pen_list.append(self.pen)
                BJpays_list.append(self.BJpays)
                seventeen_list.append(self.seventeen)
                opening_count_list.append(outcome['opening_count'])
                cards_left_list.append(outcome['cards_left'])
                spots_list.append(outcome['spots'])
                bets_list.append(outcome['player_bets'])
                double_list.append(outcome['double_down'])
                p_hands_list.append(outcome['player_hands'])
                d_hands_list.append(outcome['dealer_hand'])
                p_results_list.append(outcome['player_results'])
                d_result_list.append(outcome['dealer_result'])
                insurance_list.append(outcome['insurance'])
                wbh_list.append(outcome['player_winnings_by_hand'])
                trw_list.append(outcome['total_round_winnings'])
        
        
        df_dict = {'decks':decks_list,\
                   'pen':pen_list, 'Blackjack_pays':BJpays_list,\
                   'Dealer_seventeen':seventeen_list,'strategy':strategy_list,\
                   'opening_count':opening_count_list,\
                   'cards_left':cards_left_list, 'spots':spots_list,\
                   'bets':bets_list, 'double_down': double_list, 'player_hands':p_hands_list,\
                   'dealer_hand':d_hands_list,'player_results':p_results_list, 'dealer_result':d_result_list,\
                   'insurance':insurance_list, 'player_winnings_by_hand':wbh_list, 'total_round_winnings':trw_list}
        df = pd.DataFrame(df_dict)
        df['spread'] = df['strategy'].apply(lambda C: self.spread if C=='counter' else None)
        df = df[['decks', 'pen', 'Blackjack_pays', 'Dealer_seventeen', 'strategy','spread',\
                 'opening_count', 'cards_left', 'spots', 'bets', 'double_down','player_hands',\
                 'dealer_hand', 'player_results', 'dealer_result','insurance',\
                 'player_winnings_by_hand', 'total_round_winnings']]
        
        return df
    
    # similar function to return a DataFrame for just counter strategy
    
    def data_table_counter(self):
        
        
        
        strategy_list = ['counter']*self.num_rounds
        decks_list = [self.shoe.num]*self.num_rounds
        pen_list = [self.pen]*self.num_rounds
        BJpays_list = [self.BJpays]*self.num_rounds
        seventeen_list = [self.seventeen]*self.num_rounds
        opening_count_list = []
        cards_left_list =[]
        spots_list = []
        bets_list = []
        double_list = []
        p_hands_list = []
        d_hands_list = []
        p_results_list = []
        d_result_list = []
        insurance_list = []
        wbh_list = []
        trw_list = []
        
        minus_one_count_bet_list = [self.spread[-1][0]]*self.num_rounds
        minus_one_count_spot_list = [self.spread[-1][1]]*self.num_rounds
        
        zero_count_bet_list = [self.spread[0][0]]*self.num_rounds
        zero_count_spot_list = [self.spread[0][1]]*self.num_rounds
        
        one_count_bet_list = [self.spread[1][0]]*self.num_rounds
        one_count_spot_list = [self.spread[1][1]]*self.num_rounds
        
        two_count_bet_list = [self.spread[2][0]]*self.num_rounds
        two_count_spot_list = [self.spread[2][1]]*self.num_rounds
        
        three_count_bet_list = [self.spread[3][0]]*self.num_rounds
        three_count_spot_list = [self.spread[3][1]]*self.num_rounds
        
        four_count_bet_list = [self.spread[4][0]]*self.num_rounds
        four_count_spot_list = [self.spread[4][1]]*self.num_rounds
        
        five_count_bet_list = [self.spread[5][0]]*self.num_rounds
        five_count_spot_list = [self.spread[5][1]]*self.num_rounds
        
        six_count_bet_list = [self.spread[6][0]]*self.num_rounds
        six_count_spot_list = [self.spread[6][1]]*self.num_rounds
        
        
    
        np.random.seed(80)
        game = play(self.shoe, self.pen, 'counter', self.seventeen, self.BJpays, self.spread)
        game.fill_shoe()
        for i in range(self.num_rounds):
            outcome = game.round()[1]
            opening_count_list.append(outcome['opening_count'])
            cards_left_list.append(outcome['cards_left'])
            spots_list.append(outcome['spots'])
            bets_list.append(outcome['player_bets'])
            double_list.append(outcome['double_down'])
            p_hands_list.append(outcome['player_hands'])
            d_hands_list.append(outcome['dealer_hand'])
            p_results_list.append(outcome['player_results'])
            d_result_list.append(outcome['dealer_result'])
            insurance_list.append(outcome['insurance'])
            wbh_list.append(outcome['player_winnings_by_hand'])
            trw_list.append(outcome['total_round_winnings'])
            
        accumulated_winnings_list = np.cumsum(trw_list)
        
        
        df_dict = {'decks':decks_list,\
                   'pen':pen_list, 'Blackjack_pays':BJpays_list,\
                   'Dealer_seventeen':seventeen_list,'strategy':strategy_list,\
                   'opening_count':opening_count_list,\
                   'cards_left':cards_left_list, 'spots':spots_list,\
                   'bets':bets_list, 'double_down': double_list, 'player_hands':p_hands_list,\
                   'dealer_hand':d_hands_list,'player_results':p_results_list, 'dealer_result':d_result_list,\
                   'insurance':insurance_list, 'player_winnings_by_hand':wbh_list, 'total_round_winnings':trw_list,\
                   'accumulated_winnings':accumulated_winnings_list,\
                   '-1_count_bet':minus_one_count_bet_list,\
                   '-1_count_spot':minus_one_count_spot_list,\
                   '0_count_bet':zero_count_bet_list,\
                   '0_count_spot':zero_count_spot_list,\
                   '1_count_bet':one_count_bet_list,\
                   '1_count_spot':one_count_spot_list,\
                   '2_count_bet':two_count_bet_list,\
                   '2_count_spot':two_count_spot_list,\
                   '3_count_bet':three_count_bet_list,\
                   '3_count_spot':three_count_spot_list,\
                   '4_count_bet':four_count_bet_list,\
                   '4_count_spot':four_count_spot_list,\
                   '5_count_bet':five_count_bet_list,\
                   '5_count_spot':five_count_spot_list,\
                   '6_count_bet':six_count_bet_list,\
                   '6_count_spot':six_count_spot_list}
        
        df = pd.DataFrame(df_dict)

        
        return df
                
        

## Sampling multiple spreads for counter strategy

In [ ]:
# function to replace the second entry in a list to a 1
def to_one(ls):
    new = [i for i in ls]
    new[1] = 1
    return new

def dict_one(dic):
    return {key:to_one(dic[key]) for key in list(dic.keys())}
    

In [ ]:
spreads = {}

spreads['twelve'] = {-1:[1,1], 0:[1,1], 1:[1,2], 2:[4,2], 3:[6,2], 4:[8,2], 5:[10,2], 6:[12,2]}
spreads['twelve_1'] = dict_one(spreads['twelve'])

spreads['twenty_a'] = {-1:[1,1], 0:[1,1], 1:[2,2], 2:[6,2], 3:[10,2], 4:[13,2], 5:[17,2], 6:[20,2]}
spreads['twenty_a_1'] = dict_one(spreads['twenty_a'])

spreads['six'] = {-1:[1,1], 0:[1,1], 1:[1,2], 2:[2,1], 3:[3,2], 4:[4,2], 5:[5,2], 6:[6,2]}
spreads['six_1'] = dict_one(spreads['six'])

spreads['fifty'] = {-1:[1,1], 0:[1,1], 1:[5,1], 2:[10,1], 3:[20,2], 4:[30,2], 5:[40,2], 6:[50,2]}
spreads['fifty_1'] = dict_one(spreads['fifty'])

spreads['eight'] = {-1:[1,1], 0:[1,1], 1:[1,1], 2:[2,1], 3:[3,1], 4:[5,2], 5:[6,2], 6:[8,2]}
spreads['eight_1'] = dict_one(spreads['eight'])

spreads['eighteen'] = {-1:[1,1], 0:[1,1], 1:[3,2], 2:[6,2], 3:[9,2], 4:[12,2], 5:[15,2], 6:[18,2]}
spreads['eighteen_1'] = dict_one(spreads['eighteen'])

spreads['twenty-four'] = {-1:[1,1], 0:[1,1], 1:[4,1], 2:[8,1], 3:[14,1], 4:[16,2], 5:[18,2], 6:[24,2]}
spreads['twenty-four_1'] = dict_one(spreads['twenty-four'])

spreads['twenty_b'] = {-1:[1,1], 0:[1,1], 1:[3,2], 2:[6,2], 3:[9,2], 4:[12,2], 5:[15,2], 6:[20,2]}
spreads['twenty_b_1'] = dict_one(spreads['twenty_b'])

spreads['thirty'] = {-1:[1,1], 0:[1,1], 1:[5,1], 2:[10,1], 3:[15,1], 4:[20,2], 5:[25,2], 6:[30,2]}
spreads['thirty_1'] = dict_one(spreads['thirty'])

spreads['sixteen'] = {-1:[1,1], 0:[1,1], 1:[1,1], 2:[4,1], 3:[7,1], 4:[10,1], 5:[12,2], 6:[16,2]}
spreads['sixteen_1'] = dict_one(spreads['sixteen'])

In [ ]:
# generate product list of parameters

decks = [2,4,6]
pens = [0.5, 1, 1.5, 2]
seventeens = ['S17','H17']
BJpayouts = [[3,2],[6,5]]

product = []
for deck in decks:
    for pen in pens:
        for seventeen in seventeens:
            for BJpays in BJpayouts:
                if pen == deck:
                    continue
                else:
                    product.append((deck,pen,seventeen,BJpays))

In [ ]:
# function for collecting samples with different parameters


def collect_samples_counter(deck, pen, seventeen, BJpays, spread_name):
    counterpath = '/media/saa/My Passport/Blackjack/sim_dfs/counter'
    if not os.path.exists(counterpath):
        os.mkdir(counterpath)
        
    spreadpath = '/media/saa/My Passport/Blackjack/sim_dfs/counter/%s'%(spread_name)
    if not os.path.exists(spreadpath):
        os.mkdir(spreadpath)

    
    S = sample(pack(deck), pen, ['naive', 'cc', 'basic', 'counter'], seventeen, 1, 1000000, BJpays,\
               spreads[spread_name])

    df = S.data_table_counter()


    df.to_csv('%s/%s_%s_%s_%s_%s.csv'%(spreadpath, deck, pen, seventeen, BJpays, spread_name))

    gc.collect()

                        

In [ ]:
spread_names = list(spreads.keys())

In [ ]:
for spread_name in spread_names:    
    for params in product:
        gc.collect()
        start = time.time()
        deck, pen, seventeen, BJpays = params
        collect_samples_counter(deck, pen, seventeen, BJpays, spread_name)
        stop = time.time()
        print('sample time: ', stop-start)

## Extracting DFs for non-counter strategies

In [ ]:
# create product list to align in 
# same order of game parameters as samples were originally obtained

decks = [2,4,6]
pens = [0.5, 1, 1.5, 2]
seventeens = ['S17','H17']
BJpayouts = [[3,2],[6,5]]

product = []
j = 0
for deck in decks:
    for pen in pens:
        for seventeen in seventeens:
            for BJpays in BJpayouts:
                if pen == deck:
                    continue
                else:
                    product.append((j,deck,pen,BJpays,seventeen))
                    j += 1

In [ ]:
# function for creating DFs for each param product and strat

def gen_DF(strat):
    for param in product:
        j, deck, pen, BJpays, seventeen = param
        read_path = '/media/saa/My Passport/Blackjack/BlackjackEngine/1000000_samples/dataframes/%s__%s_decks_pen_%s_BJpays_%s_dealerplays_%s_1000000_samples.csv'%(j,deck,pen,BJpays,seventeen)
        df = pd.read_csv(read_path)
        df.drop('Unnamed: 0', axis=1, inplace=True)
        df = df[df['strategy']==strat]
        df.reset_index(drop=True, inplace=True)
        write_path = '/media/saa/My Passport/Blackjack/sim_dfs/%s/%s_%s_%s_%s_%s.csv'%(strat,deck,pen,seventeen, BJpays, strat)
        df.to_csv(write_path)
        print('completed ', param)

In [ ]:
gen_DF('basic')

In [ ]:
gen_DF('naive')

In [ ]:
gen_DF('cc')

## Obtaining smaller sample for GitHub

To illustrate the collected samples, we create "shorter" dataframes (each with 25,000 rounds) for a 6-deck, 1 penetration, dealer plays H17, blackjack pays 3:2 game played with the following strategies:
* naive
* copycat
* basic
* counter with `'twelve'` spread
* counter with `'twenty_a'` spread
These files are small enough to upload to the GitHub repository for this project.

In [ ]:
for strat in ['naive','cc','basic']:
    df = pd.read_csv('/media/saa/My Passport/Blackjack/sim_dfs/%s/6_1_H17_[3, 2]_%s.csv'%(strat,strat))

    df = df.iloc[:25000,:]
    df.to_csv('short_samples/6_1_H17_[3, 2]_%s_25k.csv'%(strat))
    
for spread_name in ['twelve','twenty_a']:
    df = pd. read_csv('/media/saa/My Passport/Blackjack/sim_dfs/counter/%s/6_1_H17_[3, 2]_%s.csv'%(spread_name,spread_name))
    
    df = df.iloc[:25000,:]
    df.to_csv('short_samples/6_1_H17_[3, 2]_%s_25k.csv'%(spread_name))